In [2]:
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
import re
from sklearn.linear_model import LinearRegression
import requests
from bs4 import BeautifulSoup
from collections import Counter
import os
import requests
import shutil
import matplotlib.pyplot as plt
import math
import requests
from sklearn.model_selection import KFold
import numpy as np
from sklearn.linear_model import Ridge

pd.options.mode.chained_assignment = None  # default='warn'


# Choose Dataset

In [3]:
df = pd.read_csv("train_final.csv")

# Split by Type

In [4]:
##### print(df.shape)

df_homes = df[df["room_type"]=="Entire home/apt"]
df_homes = df_homes[df_homes["property_type"]!="Houseboat"]
df_homes = df_homes[df_homes["property_type"]!="Boat"]


df_private_room = df[(df["room_type"]=="Private room") | (df["room_type"]=="Hotel room")]
datasets_linear_regression = [(df_homes,"Homes"),(df_private_room,"Private Rooms")]

print(df_homes.shape)
print(df_private_room.shape)

(3913, 3739)
(1476, 3739)


In [22]:
np.square(df_homes["price"]-df_homes["price"].mean()).sum()


np.float64(44141497.05298185)

# Create ranking list of features

In [5]:
ranking_features_linear_regression = ['accommodates',
 'bedrooms',
 'beds',
 'bathrooms',
 'sqm',
 'sound_bang_olufsen',
 'oven_beko',
 'property_type_Houseboat',
 'pool_view',
 'outdoor_pool_private',
 'property_type_Boat',
 'stove_whirlpool',
 'neighbourhood_cleansed_Haselhorst',
 'parking_free_residential_garage',
 'comments_tfidf_family',
 'exercise_equipment_workout_bench',
 'gym_private',
 'neighbourhood_cleansed_Forst Grunewald',
 'calculated_host_listings_count_private_rooms',
 'gym_private_in_building',
 'refrigerator_miele',
 'oven_electrolux',
 'comments_tfidf_room',
 'stove_electrolux',
 'oven_miele',
 'neighbourhood_cleansed_Hellersdorf-Süd',
 'comments_tfidf_kind',
 'stove_gaggenau',
 'stove_miele',
 'property_type_Shared room in rental unit',
 'estimated_occupancy_l365d',
 'property_type_Shared room in bed and breakfast',
 'property_type_Shared room in hostel',
 'parking_free_carport',
 'distance_to_center',
 'refrigerator_bomann',
 'baby_bath',
 'property_type_Entire home',
 'comments_tfidf_spacious',
 'property_type_Shared room in serviced apartment',
 'neighbourhood_cleansed_Allende-Viertel',
 'neighbourhood_cleansed_Neu-Hohenschönhausen Süd',
 'minimum_nights',
 'baby_monitor',
 'neighbourhood_cleansed_Friedrichsfelde Süd',
 'neighbourhood_cleansed_Schmöckwitz/Karolinenhof/Rauchfangswerder',
 'comments_tfidf_large',
 'neighbourhood_cleansed_Marzahn-Mitte',
 'property_type_Private room in guest suite',
 'exercise_equipment_treadmill',
 'comments_tfidf_cozy',
 'property_type_Private room in vacation home',
 'property_type_Private room in townhouse',
 'oven_whirlpool',
 'comments_tfidf_apartment great',
 'property_type_Private room in home',
 'neighbourhood_cleansed_Heerstraße Nord',
 'neighbourhood_cleansed_Charlottenburg Nord',
 'description_tfidf_bedroom',
 'sound_sony',
 'property_type_Entire townhouse',
 'refrigerator_panasonic',
 'comments_tfidf_fantastic',
 'property_type_Casa particular',
 'sound_sonos',
 'neighbourhood_cleansed_Ost 2',
 'description_tfidf_room',
 'desert_view',
 'sound_onkyo',
 'neighbourhood_cleansed_Gropiusstadt',
 'description_tfidf_family',
 'neighbourhood_cleansed_Buch',
 'neighbourhood_cleansed_Kölln. Vorstadt/Spindlersf.',
 'refrigerator_beko',
 'review_scores_cleanliness',
 'neighbourhood_cleansed_Köpenick-Süd',
 'property_type_Private room in villa',
 'property_type_Room in serviced apartment',
 'description_tfidf_spacious',
 'property_type_Private room in rental unit',
 'exercise_equipment_elliptical',
 'property_type_Room in hostel',
 'number_of_reviews_l30d',
 'property_type_Private room in casa particular',
 'neighbourhood_cleansed_Lankwitz',
 'neighbourhood_cleansed_West 4',
 'comments_tfidf_welcome',
 'parking_paid_garage',
 'sound_pioneer',
 'sound_marshall',
 'comments_tfidf_station',
 'refrigerator_gorenje',
 'neighbourhood_cleansed_Falkenhagener Feld',
 'neighbourhood_cleansed_Brunnenstr. Süd',
 'host_total_listings_count',
 'neighbourhood_cleansed_Buckow Nord',
 'refrigerator_liebherr',
 'neighbourhood_cleansed_MV 2',
 'comments_tfidf_love',
 'oven_gaggenau',
 'property_type_Cave',
 'property_type_Private room in condo',
 'reviews_per_month',
 'neighbourhood_cleansed_Bohnsdorf',
 'property_type_Entire loft',
 'gym',
 'neighborhood_overview_tfidf_museum',
 'neighbourhood_cleansed_Regierungsviertel',
 'property_type_Private room in loft',
 'sound_samsung',
 'description_tfidf_living',
 'stove_teka',
 'property_type_Dome',
 'outdoor_kitchen',
 'wifi_speed_mbps',
 'neighbourhood_cleansed_Ost 1',
 'sound_denon',
 'neighbourhood_cleansed_West 5',
 'number_of_reviews',
 'gym_private_nearby',
 'property_type_Private room in pension',
 'property_type_Entire serviced apartment',
 'comments_tfidf_bus',
 'air_conditioning',
 'refrigerator_sharp',
 'clothing_storage_walk_in',
 'comments_tfidf_space',
 'refrigerator_amazon',
 'neighbourhood_cleansed_Nord 1',
 'refrigerator_zanussi',
 'calculated_host_listings_count_shared_rooms',
 'neighbourhood_cleansed_Johannisthal',
 'neighbourhood_cleansed_Rummelsburger Bucht',
 'high_chair',
 'comments_tfidf_great location',
 'property_type_Private room in bungalow',
 'neighbourhood_cleansed_Malchow, Wartenberg und Falkenberg',
 'has_sound_system',
 'comments_tfidf_beautiful',
 'neighbourhood_cleansed_Nord 2',
 'exercise_equipment_free_weights',
 'neighbourhood_cleansed_Schloßstr.',
 'comments_tfidf_help',
 'neighbourhood_cleansed_Blankenburg/Heinersdorf/Märchenland',
 'sound_yamaha',
 'neighbourhood_cleansed_Frankfurter Allee Süd',
 'description_tfidf_design',
 'refrigerator_vestel',
 'refrigerator_amica',
 'neighbourhood_cleansed_Brunsbütteler Damm',
 'description_tfidf_fully equip',
 'outdoor_pool_shared',
 'property_type_Private room in bed and breakfast',
 'neighbourhood_cleansed_Osloer Straße',
 'calculated_host_listings_count',
 'neighbourhood_cleansed_Köpenick-Nord',
 'neighbourhood_cleansed_Baumschulenweg',
 'refrigerator_siemens',
 'comments_tfidf_near',
 'neighbourhood_cleansed_Tempelhof',
 'exercise_equipment_yoga_mat',
 'neighbourhood_cleansed_Schmargendorf',
 'desc_sentiment',
 'neighbourhood_cleansed_Rudow',
 'neighbourhood_cleansed_Parkviertel',
 'neighbourhood_cleansed_Lichtenrade',
 'review_scores_rating',
 'baby_safety',
 'stove_zanussi',
 'sound_panasonic',
 'tv_size_inch',
 'property_type_Private room in guesthouse',
 'neighbourhood_cleansed_West 3',
 'ps4',
 'host_listings_count',
 'crib',
 'neighbourhood_cleansed_Gatow / Kladow',
 'xboxone',
 'comments_tfidf_minute',
 'has_hottub',
 'property_type_Entire villa',
 'neighbourhood_cleansed_Altstadt-Kietz',
 'neighborhood_overview_tfidf_immediate',
 'city_skyline_view',
 'neighbourhood_cleansed_Ostpreußendamm',
 'neighbourhood_cleansed_Hellersdorf-Nord',
 'comments_tfidf_recommend place',
 'refrigerator_lg',
 'stove_bosch',
 'neighbourhood_group_cleansed_Reinickendorf',
 'comments_tfidf_short',
 'children_books_and_toys',
 'streaming_apple_tv',
 'neighbourhood_cleansed_Oberschöneweide',
 'description_tfidf_fully',
 'neighbourhood_cleansed_Britz',
 'pool_table',
 'sound_bose',
 'comments_tfidf_shop',
 'property_type_Private room',
 'coffee_nespresso',
 'refrigerator_gaggenau',
 'desc_len',
 'property_type_Private room in hostel',
 'neighbourhood_cleansed_Prenzlauer Berg Südwest',
 'neighbourhood_cleansed_Siemensstadt',
 'host_acceptance_rate_numeric',
 'neighbourhood_cleansed_Teltower Damm',
 'neighbourhood_cleansed_Altglienicke',
 'neighbourhood_cleansed_Zehlendorf  Südwest',
 'description_tfidf_quiet',
 'refrigerator_smeg',
 'neighbourhood_cleansed_Fennpfuhl',
 'parking_paid_lot',
 'neighborhood_overview_tfidf_lively',
 'property_type_Entire guesthouse',
 'game_item_games',
 'neighbourhood_cleansed_Blankenfelde/Niederschönhausen',
 'comments_tfidf_metro',
 'neighbourhood_cleansed_Grünau',
 'ps2',
 'neighbourhood_cleansed_Weißensee Ost',
 'comments_tfidf_excellent',
 'neighbourhood_cleansed_Brunnenstr. Nord',
 'clothing_storage_closet',
 'has_tv',
 'description_tfidf_space',
 'neighbourhood_cleansed_Buckow',
 'has_game_console',
 'property_type_Camper/RV',
 'comments_tfidf_min',
 'dryer_in_building',
 'neighbourhood_cleansed_Schönholz/Wilhelmsruh/Rosenthal',
 'refrigerator_ikea',
 'review_scores_location',
 'neighbourhood_cleansed_West 2',
 'oven_ikea',
 'neighbourhood_cleansed_Alexanderplatz',
 'neighborhood_overview_tfidf_mitte',
 'neighbourhood_cleansed_Müggelheim',
 'has_exercise_equipment',
 'neighbourhood_cleansed_Volkspark Wilmersdorf',
 'description_tfidf_large',
 'streaming_amazon_prime_video',
 'property_type_Private room in boat',
 'neighbourhood_cleansed_Rixdorf',
 'comments_tfidf_public transport',
 'washer_paid',
 'pets_allowed',
 'has_streaming_provider',
 'availability_30',
 'streaming_netflix',
 'ps3',
 'property_type_Private room in serviced apartment',
 'comments_tfidf_responsive',
 'refrigerator_bosch',
 'neighbourhood_cleansed_Köllnische Heide',
 'refrigerator_whirlpool',
 'description_tfidf_heart',
 'oven_bosch',
 'neighbourhood_cleansed_Rahnsdorf/Hessenwinkel',
 'parking_paid',
 'description_tfidf_minute',
 'has_balcony',
 'outdoor_furniture',
 'neighbourhood_cleansed_Friedenau',
 'comments_tfidf_amazing',
 'neighbourhood_cleansed_Mierendorffplatz',
 'neighbourhood_group_cleansed_Mitte',
 'has_coffee_maker',
 'comments_tfidf_distance',
 'neighbourhood_group_cleansed_Steglitz - Zehlendorf',
 'neighbourhood_cleansed_Adlershof',
 'has_dryer',
 'description_tfidf_living room',
 'neighbourhood_cleansed_Karow',
 'neighbourhood_cleansed_Wilhelmstadt',
 'neighbourhood_cleansed_Karlshorst',
 'description_tfidf_small',
 'neighbourhood_cleansed_Schöneberg-Nord',
 'parking_on_premises',
 'kitchen_item_baking_sheet',
 'comments_tfidf_cafe',
 'has_bath_bed_game_items',
 'neighbourhood_cleansed_Biesdorf',
 'neighbourhood_cleansed_Prenzlauer Berg Ost',
 'description_tfidf_bahn',
 'streaming_disney_plus',
 'outdoor_dining_area',
 'oven_smeg',
 'neighborhood_overview_tfidf_location',
 'neighbourhood_cleansed_Neu Lichtenberg',
 'property_type_Entire condo',
 'neighbourhood_cleansed_Schloß Charlottenburg',
 'description_tfidf_area',
 'gym_shared_in_building',
 'stove_ikea',
 'has_bathtub',
 'description_tfidf_high',
 'bath_item_conditioner',
 'oven_siemens',
 'has_outdoor',
 'comments_tfidf_bahn',
 'has_refrigerator',
 'outdoor_playground',
 'stove_induction',
 'comments_tfidf_great host',
 'neighbourhood_group_cleansed_Spandau',
 'property_type_Entire cottage',
 'property_type_Island',
 'description_tfidf_equip',
 'neighbourhood_cleansed_MV 1',
 'neighbourhood_cleansed_Tiergarten Süd',
 'has_oven',
 'comments_tfidf_transport',
 'neighbourhood_cleansed_Dammvorstadt',
 'bed_item_bed_linens',
 'neighbourhood_group_cleansed_Treptow - Köpenick',
 'description_tfidf_min',
 'neighbourhood_cleansed_Buchholz',
 'has_stove',
 'neighbourhood_cleansed_Barstraße',
 'latitude',
 'has_heating',
 'refrigerator_neff',
 'neighbourhood_cleansed_Karl-Marx-Allee-Nord',
 'host_response_rate_numeric',
 'neighbourhood_cleansed_Kurfürstendamm',
 'parking_off_premises',
 'neighbourhood_cleansed_Grunewald',
 'neighborhood_overview_tfidf_popular',
 'property_type_Entire cabin',
 'property_type_Room in hotel',
 'neighbourhood_cleansed_Moabit Ost',
 'comments_tfidf_floor',
 'neighbourhood_cleansed_Alt-Lichtenberg',
 'wii',
 'gym_shared',
 'host_about_tfidf_apartment',
 'neighbourhood_cleansed_Schöneberg-Süd',
 'refrigerator_haier',
 'neighbourhood_cleansed_Kantstraße',
 'description_tfidf_right',
 'property_type_Room in boutique hotel',
 'neighbourhood_cleansed_Niederschöneweide',
 'neighbourhood_cleansed_Karl-Marx-Allee-Süd',
 'neighborhood_overview_tfidf_gallery',
 'dryer_paid',
 'property_type_Entire chalet',
 'refrigerator_samsung',
 'comments_tfidf_night',
 'parking_free_driveway',
 'refrigerator_severin',
 'stove_siemens',
 'neighborhood_overview_tfidf_berlin',
 'neighbourhood_group_cleansed_Neukölln',
 'property_type_Entire rental unit',
 'private_entrance',
 'comments_tfidf_experience',
 'neighbourhood_cleansed_Pankow Süd',
 'oven_aeg',
 'gym_shared_nearby',
 'description_tfidf_apartment',
 'stove_aeg',
 'neighbourhood_cleansed_südliche Luisenstadt',
 'neighbourhood_cleansed_Albrechtstr.',
 'description_tfidf_modern',
 'neighbourhood_cleansed_Prenzlauer Berg Nord',
 'housekeeping_included',
 'ping_pong_table',
 'comments_tfidf_big',
 'neighbourhood_cleansed_Frankfurter Allee Nord',
 'parking_paid_street',
 'refrigerator_electrolux',
 'host_about_tfidf_time',
 'neighbourhood_cleansed_Alt  Treptow',
 'neighbourhood_cleansed_Reuterstraße',
 'sound_teufel',
 'description_tfidf_station',
 'oven_zanussi',
 'neighbourhood_cleansed_West 1',
 'neighbourhood_cleansed_Zehlendorf  Nord',
 'comments_tfidf_warm',
 'instant_bookable',
 'refrigerator_aeg',
 'bath_item_shampoo',
 'comments_tfidf_definitely recommend',
 'neighbourhood_cleansed_Spandau Mitte',
 'exercise_equipment_stationary_bike',
 'neighbourhood_group_cleansed_Lichtenberg',
 'comments_tfidf_supermarket',
 'host_greets_you',
 'housekeeping_available_at_cost',
 'bath_item_body_soap',
 'neighborhood_overview_tfidf_art',
 'has_washer',
 'neighbourhood_cleansed_Prenzlauer Berg Süd',
 'backyard_shared',
 'comments_tfidf_public',
 'neighbourhood_group_cleansed_Marzahn - Hellersdorf',
 'stove_smeg',
 'neighbourhood_cleansed_Tempelhofer Vorstadt',
 'comments_tfidf_house',
 'neighbourhood_cleansed_Drakestr.',
 'neighbourhood_cleansed_Düsseldorfer Straße',
 'neighbourhood_cleansed_Alt-Hohenschönhausen Süd',
 'neighborhood_overview_tfidf_away',
 'neighbourhood_cleansed_Heerstrasse',
 'oven_neff',
 'sound_jbl',
 'description_tfidf_berlin',
 'neighbourhood_cleansed_Prenzlauer Berg Nordwest',
 'workspace',
 'neighbourhood_cleansed_Halensee',
 'comments_tfidf_neighbourhood',
 'has_parking',
 'stove_beko',
 'neighbourhood_cleansed_Marienfelde',
 'host_about_tfidf_fully',
 'washer_in_building',
 'beach_view',
 'backyard_fully_fenced',
 'review_scores_accuracy',
 'neighbourhood_cleansed_Moabit West',
 'clothing_storage_wardrobe',
 'property_type_Entire place',
 'host_response_time_numeric',
 'neighbourhood_cleansed_Marzahn-Süd',
 'property_type_Entire guest suite',
 'property_type_Entire vacation home',
 'description_tfidf_walk',
 'luggage_dropoff_allowed',
 'long_term_stays_allowed',
 'sound_lg',
 'neighborhood_overview_tfidf_supermarket',
 'neighbourhood_group_cleansed_Charlottenburg-Wilm.',
 'host_in_berlin',
 'has_clothing_storage',
 'comments_tfidf_place stay',
 'neighbourhood_cleansed_Hakenfelde',
 'parking_free_garage',
 'neighbourhood_cleansed_Pankow Zentrum',
 'neighborhood_overview_tfidf_numerous',
 'property_type_Tiny home',
 'has_central_heating',
 'sound_philips',
 'comments_tfidf_coffee',
 'neighbourhood_cleansed_nördliche Luisenstadt',
 'host_identity_verified',
 'neighbourhood_cleansed_Mahlsdorf',
 'comments_tfidf_street',
 'neighbourhood_group_cleansed_Tempelhof - Schöneberg',
 'neighbourhood_cleansed_Weißensee',
 'parking_free_street',
 'neighborhood_overview_tfidf_nearby',
 'neighbourhood_cleansed_Neue Kantstraße',
 'neighbourhood_cleansed_Wiesbadener Straße',
 'garden_view',
 'comments_tfidf_explore',
 'neighbourhood_cleansed_Otto-Suhr-Allee',
 'stove_gas',
 'has_backyard',
 'comments_tfidf_train',
 'property_type_Private room in houseboat',
 'neighbourhood_cleansed_Wedding Zentrum',
 'neighbourhood_cleansed_Alt-Hohenschönhausen Nord',
 'comments_tfidf_locate',
 'neighbourhood_cleansed_Westend',
 'neighbourhood_cleansed_Mariendorf',
 'pour_over_coffee',
 'comments_tfidf_great stay',
 'neighbourhood_cleansed_Schillerpromenade',
 'host_is_superhost',
 'neighbourhood_cleansed_Friedrichsfelde Nord',
 'comments_tfidf_small',
 'ps5',
 'neighbourhood_cleansed_Kaulsdorf',
 'stove_electric',
 'host_about_tfidf_quality',
 'neighbourhood_cleansed_Südliche Friedrichstadt',
 'property_type_Entire bungalow',
 'neighbourhood_cleansed_Friedrichshagen',
 'description_tfidf_balcony',
 'coffee_espresso',
 'coffee_french_plus',
 'neighbourhood_cleansed_Frankfurter Allee Süd FK',
 'comments_tfidf_bar',
 'has_wifi',
 'neighbourhood_cleansed_Plänterwald',
 'neighbourhood_group_cleansed_Friedrichshain-Kreuzberg',
 'host_about_tfidf_bed',
 'stove_neff',
 'neighbourhood_group_cleansed_Pankow',
 'neighbourhood_cleansed_Helmholtzplatz',
 'comments_tfidf_bathroom',
 'nintendoswitch',
 'neighbourhood_cleansed_Neuköllner Mitte/Zentrum',
 'property_type_Room in aparthotel',
 'neighborhood_overview_tfidf_area',
 'description_tfidf_floor',
 'comments_tfidf_building',
 'description_tfidf_kitchen',
 'host_about_tfidf_like',
 'description_tfidf_guest',
 'neighborhood_overview_tfidf_bahn',
 'host_about_tfidf_family',
 'comments_tfidf_minute walk',
 'host_about_tfidf_year',
 'comments_tfidf_location great',
 'maximum_nights',
 'neighborhood_overview_tfidf_beautiful',
 'description_tfidf_house',
 'sentiment_polarity',
 'neighborhood_overview_tfidf_shopping',
 'neighborhood_overview_tfidf_park',
 'host_about_tfidf_high',
 'neighborhood_overview_tfidf_place',
 'description_tfidf_beautiful',
 'description_tfidf_studio',
 'comments_tfidf_away',
 'comments_tfidf_communication',
 'comments_tfidf_amenity',
 'host_about_tfidf_experience',
 'comments_tfidf_central',
 'comments_tfidf_bed',
 'description_tfidf_enjoy',
 'description_tfidf_location',
 'host_about_tfidf_meet',
 'comments_tfidf_sleep',
 'neighborhood_overview_tfidf_city',
 'comments_tfidf_wonderful',
 'comments_tfidf_book',
 'neighborhood_overview_tfidf_apartment locate',
 'comments_tfidf_local',
 'description_tfidf_offer',
 'neighborhood_overview_tfidf_cafe',
 'neighborhood_overview_tfidf_right',
 'host_about_tfidf_high quality',
 'neighborhood_overview_tfidf_apartment',
 'host_about_tfidf_equip',
 'description_tfidf_home',
 'neighborhood_overview_tfidf_corner',
 'comments_tfidf_little',
 'comments_tfidf_people',
 'neighborhood_overview_tfidf_transport',
 'comments_tfidf_come',
 'comments_tfidf_overall',
 'comments_tfidf_apartment clean',
 'host_about_tfidf_live',
 'host_about_tfidf_home',
 'comments_tfidf_bit',
 'description_tfidf_perfect',
 'host_about_tfidf_team',
 'review_scores_communication',
 'longitude_host_neighbourhood',
 'description_tfidf_live',
 'neighborhood_overview_tfidf_good',
 'neighborhood_overview_tfidf_public',
 'description_tfidf_private',
 'description_tfidf_bed',
 'id',
 'neighborhood_overview_tfidf_trendy',
 'neighborhood_overview_tfidf_min',
 'description_tfidf_close',
 'host_about_tfidf_design',
 'neighborhood_overview_tfidf_club',
 'host_about_tfidf_fully equip kitchen',
 'comments_tfidf_kitchen',
 'description_tfidf_good',
 'neighborhood_overview_tfidf_kreuzberg',
 'host_about_tfidf_equip kitchen',
 'host_about_tfidf_good',
 'host_about_tfidf_fully equip',
 'host_about_tfidf_welcome',
 'host_about_tfidf_digital',
 'description_tfidf_comfortable',
 'neighborhood_overview_tfidf_neighborhood',
 'host_about_tfidf_travel',
 'neighborhood_overview_tfidf_walk',
 'review_scores_value',
 'host_about_tfidf_world',
 'latitude_host_neighbourhood',
 'description_tfidf_welcome',
 'neighborhood_overview_tfidf_restaurant',
 'description_tfidf_flat',
 'neighborhood_overview_tfidf_offer',
 'neighborhood_overview_tfidf_close',
 'neighborhood_overview_tfidf_famous',
 'comments_tfidf_enjoy',
 'neighborhood_overview_tfidf_distance',
 'comments_tfidf_nearby',
 'host_about_tfidf_love travel',
 'host_about_tfidf_stay',
 'host_about_tfidf_people',
 'neighborhood_overview_tfidf_district',
 'host_about_tfidf_suite',
 'comments_tfidf_good location',
 'review_scores_checkin',
 'comments_tfidf_trip',
 'comments_tfidf_provide',
 'neighborhood_overview_tfidf_market',
 'host_about_tfidf_check',
 'comments_tfidf_access',
 'comments_tfidf_check',
 'comment_len',
 'description_tfidf_cozy',
 'description_tfidf_place',
 'comments_tfidf_visit',
 'description_tfidf_building',
 'host_about_tfidf_love',
 'description_tfidf_bathroom',
 'host_about_tfidf_kitchen',
 'neighborhood_overview_tfidf_platz',
 'host_about_tfidf_day',
 'host_about_tfidf_work',
 'neighborhood_overview_tfidf_bar',
 'host_about_tfidf_feature',
 'description_tfidf_shower',
 'comments_tfidf_want',
 'comments_tfidf_tram',
 'host_about_tfidf_life',
 'comments_tfidf_work',
 'neighborhood_overview_tfidf_station',
 'neighborhood_overview_tfidf_walking distance',
 'host_about_tfidf_live berlin',
 'host_about_tfidf_new',
 'comments_tfidf_home',
 'longitude',
 'comments_tfidf_flat',
 'host_about_tfidf_europe',
 'neighborhood_overview_tfidf_shop',
 'neighborhood_overview_tfidf_walking',
 'comment_sentiment',
 'description_tfidf_central',
 'comments_tfidf_enjoy stay',
 'neighborhood_overview_tfidf_minute',
 'host_about_tfidf_forward',
 'host_about_tfidf_look forward',
 'neighborhood_overview_tfidf_small',
 'description_tfidf_need',
 'comments_tfidf_place great',
 'description_tfidf_stay',
 'neighborhood_overview_tfidf_friedrichshain',
 'neighborhood_overview_tfidf_berg',
 'host_about_tfidf_free',
 'neighborhood_overview_tfidf_minute walk',
 'host_about_tfidf_share',
 'neighborhood_overview_tfidf_locate',
 'host_about_tfidf_city',
 'description_tfidf_locate',
 'host_about_tfidf_easy',
 'neighborhood_overview_tfidf_street',
 'neighborhood_overview_tfidf_prenzlauer berg',
 'neighborhood_overview_tfidf_center',
 'host_about_tfidf_guest',
 'host_about_tfidf_hi',
 'comments_tfidf_airbnb',
 'comments_tfidf_shower',
 'comments_tfidf_quick',
 'host_about_tfidf_happy',
 'comments_tfidf_transportation',
 'comments_tfidf_thing',
 'neighborhood_overview_tfidf_quiet',
 'neighborhood_overview_tfidf_prenzlauer',
 'host_about_tfidf_feel home',
 'neighborhood_overview_tfidf_reach',
 'host_about_tfidf_business',
 'host_about_tfidf_enjoy',
 'comments_tfidf_stay berlin',
 'host_about_tfidf_email',
 'comments_tfidf_cool',
 'comments_tfidf_look',
 'host_about_tfidf_unique',
 'host_about_tfidf_place',
 'host_about_tfidf_berlin',
 'host_about_tfidf_room',
 'comments_tfidf_door',
 'neighborhood_overview_tfidf_house',
 'host_about_tfidf_look',
 'host_about_tfidf_feel',
 'description_tfidf_bar',
 'comments_tfidf_convenient',
 'description_tfidf_away',
 'host_about_tfidf_access',
 'comments_tfidf_neighborhood',
 'description_tfidf_restaurant',
 'description_tfidf_city',
 'comments_tfidf_right',
 'comments_tfidf_day',
 'description_tfidf_people',
 'host_about_tfidf_host',
 'comments_tfidf_great place']

# Ridge-Regression 

In [7]:
# Choose Number of features to be used from ranking_features_linear_regression

In [6]:
feature_limit = 251

In [ ]:
alphas = [0,1/2,1,2,4,8,16,32,64,128,256,512,1024,2048,4096,10000]
kf = KFold(n_splits=5, shuffle=True, random_state=42)
# To store best results per iteration
best_results = []

In [16]:
for i in range(1, feature_limit,10):  # 25 iterations

    df_cv_scores = []
    current_cv_score = 0
    
    for df_curr,df_name in datasets_linear_regression:
        feature_columns = ranking_features_linear_regression[:i]

        alpha_results = []


        for alpha in alphas:
            fold_train_mse_list = []
            fold_val_mse_list = []
            r_squared_scores = []

            for fold, (train_idx, val_idx) in enumerate(kf.split(df_curr)):
     
                 
                 df_train = df_curr.iloc[train_idx]
                 df_val = df_curr.iloc[val_idx]

                
                 existing_features = [col for col in feature_columns if col in df_train.columns]
                 
                 X_train = df_train[existing_features]
                 X_val = df_val[existing_features]
                 
                
                 y_train = df_train["price"]
                 y_val = df_val["price"]


                
                 #### Feature Preprocessing 
                 #### This has to be inside CV loop
                 ##########################



                 # FillNaN
                 for  column in  X_train.columns[X_train.isnull().any() | X_val.isnull().any() ] :
                     
                      mean = X_train[column].mean()
                      X_train[column] = X_train[column].fillna(mean)                   
                      X_val[column]   =  X_val[column].fillna(mean)
                      
                 # Dont Standardize Numeric since worse results (ofc with changed alphas)
                 

                 # for  column in  X_train.columns:
                 #     mean = X_train[column].mean()
                 #     std = X_train[column].std()
                 #     X_train[column] -= mean
                 #     X_train[column] /= std if X_train[column].std() != 0 else 1
                     
                 #     X_val[column] -= mean
                 #     X_val[column] /= std if X_train[column].std() != 0 else 1
                      
                 ################

                 ridge = Ridge(alpha=alpha)


                 ridge.fit(X_train, y_train)

                # Print parameters
                
                 # if (fold == 0 and alpha==8) :
                 #    coef_df = pd.DataFrame({
                 #        "Feature": X_train.columns,
                 #        "Coefficient": ridge.coef_
                 #    })
                 #    print(coef_df.to_string(index=False))

                 y_train_pred = ridge.predict(X_train)
                 y_val_pred = ridge.predict(X_val)

                 train_mse = mean_squared_error(y_train, y_train_pred)
                 val_mse = mean_squared_error(y_val, y_val_pred)

            

                 # Calculate R^2
                 y_train_mean = np.ones(len(y_val))*y_train.mean()
                 r_squared = 1-val_mse/mean_squared_error(y_val, y_train_mean) 
                
                 fold_train_mse_list.append(train_mse)
                 fold_val_mse_list.append(val_mse)
                 r_squared_scores.append(r_squared)
                 
        
            
            iteration_train_score = np.mean(fold_train_mse_list)
            iteration_val_score = np.mean(fold_val_mse_list)
            r_score_mean = np.mean(fold_val_mse_list)
            alpha_results.append({
            "Iteration": i,
            "Dataset": df_name,   
            "Alpha": alpha,
            "Train_MSE": iteration_train_score,
            "Validation_MSE": iteration_val_score,
            "rsquared": r_squared    

           } )

        # Select the best alpha with lowest Validation MSE for this iteration
        best_alpha_result = min(alpha_results, key=lambda x: x["Validation_MSE"])
        best_results.append(best_alpha_result)
        print(f"Iteration {i}: Best Alpha = {best_alpha_result['Alpha']}, "
            f"Dataset:  {df_name}, "
            f"Train CV Score = {best_alpha_result['Train_MSE']:.4f}, "
            f"Validation CV Score = {best_alpha_result['Validation_MSE']:.4f},"
            f"R^2 = {best_alpha_result['rsquared']:.4f}" )

Iteration 1: Best Alpha = 2.0, Dataset:  Homes, Train CV Score = 6079.3962, Validation CV Score = 6094.1711,R^2 = 0.2737
Iteration 1: Best Alpha = 2.0, Dataset:  Private Rooms, Train CV Score = 3590.0904, Validation CV Score = 3599.4857,R^2 = 0.1323
Iteration 11: Best Alpha = 2.0, Dataset:  Homes, Train CV Score = 4774.6159, Validation CV Score = 5141.4281,R^2 = 0.3404
Iteration 11: Best Alpha = 2.0, Dataset:  Private Rooms, Train CV Score = 3561.4513, Validation CV Score = 3600.8162,R^2 = 0.1420
Iteration 21: Best Alpha = 2.0, Dataset:  Homes, Train CV Score = 4723.9666, Validation CV Score = 5224.2197,R^2 = 0.3169
Iteration 21: Best Alpha = 2.0, Dataset:  Private Rooms, Train CV Score = 3498.3119, Validation CV Score = 3599.0430,R^2 = 0.1110
Iteration 31: Best Alpha = 2.0, Dataset:  Homes, Train CV Score = 4623.9416, Validation CV Score = 5143.0688,R^2 = 0.3401
Iteration 31: Best Alpha = 2.0, Dataset:  Private Rooms, Train CV Score = 3295.6575, Validation CV Score = 3416.4119,R^2 = 0


KeyboardInterrupt



# Save Results as csv

In [13]:
# # Convert list of dicts to DataFrame
# df_best_results = pd.DataFrame(best_results)
# df_best_results.to_csv("results_linear_reg.csv", index=False)

In [12]:
3913/(3913+1476)*4163.5866 + 1476/(3913+1476)*3497.1579

3981.057603674151